In [1]:
import os
from json import load
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import chainer
import chainer.links as L
import chainer.functions as F
from chainer import cuda
from chainer import optimizers
from chainer import iterators
from chainer import training
from chainer.training import extensions
from chainer import datasets
from chainer.datasets import TransformDataset

from functools import partial
from chainercv import transforms

In [2]:

from chainer.datasets import LabeledImageDataset
train = LabeledImageDataset('/home/ubuntu/intern-winter-2019/dataset/simple_crop/train_data/train_label.csv',
                            '/home/ubuntu/intern-winter-2019/dataset/simple_crop/train_data/images/',dtype=np.float32)
test = LabeledImageDataset('/home/ubuntu/intern-winter-2019/dataset/simple_crop/test_data/test_label.csv',
                            '/home/ubuntu/intern-winter-2019/dataset/simple_crop/test_data/images/',dtype=np.float32)

In [3]:
cnt_0,cnt_1=0,0
for i in range(len(test)):
    if test[i][1]==0:
        cnt_0+=1
    else:
        cnt_1+=1
print(cnt_0, cnt_1)

4907 475


In [4]:
from chainer.datasets import TransformDataset

def transform(data, train='False'):
    img, label = data
    img = img.copy()

    # Color augmentation
    if train and label==0:
        img = transforms.pca_lighting(img, 76.5)

    # Random flip & crop
    if train and label==0:
        img = transforms.random_flip(img, x_random=True)
        img = transforms.random_expand(img, max_ratio=1.5)
        img = transforms.random_crop(img, (200, 200))
    img=np.array(img, dtype=np.float32)
    img=L.model.vision.vgg.prepare(img)

    return img, label

In [5]:
train_dataset = TransformDataset(train, partial(transform, train=True))

In [6]:
test_dataset = TransformDataset(test, partial(transform, train=False))

In [7]:
len(test_dataset)

5382

In [8]:
class Model(chainer.Chain):
    def __init__(self, dim=2):
        super(Model, self).__init__()
        with self.init_scope():
            self.vgg=L.VGG16Layers()
            self.fc=L.Linear(None, dim)
            
    def __call__(self, x, train=False):
        with chainer.using_config('train', train):
            h=self.vgg(x, layers=['fc7'])['fc7']
            h=F.normalize(h)*50
            y=self.fc(h)
        return y
    
class SoftMaxEntoropyLoss(chainer.Chain):
    def __init__(self, model):
        super(SoftMaxEntoropyLoss, self).__init__()
        with self.init_scope():
            self.model = model

    def __call__(self, x, t):
        y = self.model(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        summary = F.classification_summary(y, t, beta = 1.0)
        precision = summary[0]
        recall = summary[1]
        f_value = summary[2]
        precision_dict=dict(('precision_%d' % i, val) for i, val in enumerate(summary[0]))
        recall_dict=dict(('recall_%d' % i, val) for i, val in enumerate(summary[1]))
        f_value_dict=dict(('f_value_%d' % i, val) for i, val in enumerate(summary[2]))
        main_dict={'loss':loss, 'accuracy':accuracy}
        main_dict.update(precision_dict)
        main_dict.update(recall_dict)
        main_dict.update(f_value_dict)
        chainer.report(main_dict, self)
        return loss

In [9]:
# VGG16の初期化 modelの作成
vgg = L.VGG16Layers()

In [10]:
model=Model()
loss=SoftMaxEntoropyLoss(model)

In [11]:
n_epoch = 70
batchsize = 32
out_dir = './result/'
report_interval = (10, 'iteration')

train_iter = iterators.MultithreadIterator\
(train_dataset, batchsize, repeat=True, shuffle=True)
test_iter = iterators.MultithreadIterator\
(test_dataset, batchsize, repeat=False, shuffle=False)

In [12]:
#fine tuningなのでMomentumSDG
optimizer = chainer.optimizers.MomentumSGD(lr=1e-7)
optimizer.setup(loss)

In [13]:
#device=0でGPU, device=-1でCPU
updater = training.StandardUpdater(train_iter, optimizer, device=0)

In [14]:
trainer = training.Trainer(updater, (n_epoch, 'epoch'), out=out_dir)
trainer.extend(extensions.ExponentialShift('lr', 10), trigger=(10, 'epoch'))
trainer.extend(extensions.LogReport(trigger=report_interval))
trainer.extend(extensions.PrintReport(['epoch', 
                                       'iteration', 
                                       'main/loss',
                                       'main/accuracy', 
                                       'main/precision_0', 
                                       'main/precision_1',
                                       'main/recall_0',
                                       'main/recall_1']),
               trigger=report_interval)
trainer.extend(extensions.PlotReport(y_keys='main/loss', trigger=report_interval, file_name='vgg16_loss_plot.png'))
trainer.extend(extensions.PlotReport(y_keys='main/accuracy', trigger=report_interval, file_name='vgg16_accuracy_plot.png'))

In [15]:
trainer.run()

epoch       iteration   main/loss   main/accuracy  main/precision_0  main/precision_1  main/recall_0  main/recall_1
0           10          0.924821    0.375          0.0131097         0.960806          nan            0.374032       
0           20          0.882823    0.3875         0.0571507         0.962408          nan            0.369607       
0           30          0.812998    0.46875        0.0566383         0.962943          nan            0.458452       
0           40          0.883447    0.39375        0.0178122         0.984118          nan            0.391438       
0           50          0.894616    0.390625       0.0197955         0.926491          0.25           0.395591       
0           60          0.828036    0.459375       0.0477908         0.980451          nan            0.447561       
1           70          0.799369    0.475          0.0527267         0.964282          nan            0.466895       
1           80          0.74746     0.528125       0.04854